In [1]:
from libs.libraries import *
from simulation_files.simulation_env import *
from simulation_files.mobility.mobility_models import *
#from mobility_graphics import *

scikit-learn has been installed.
All packages are up to date.


In [ ]:
print("\nSimulation Parameters:\n")
simulation_params()


Simulation Parameters:

earth_radius_km (km): 6371.0
ORIGIN (coordinates (lat, long)): (48.81778, 2.23333)
DATA_SOURCE_LINK (URL): https://files.data.gouv.fr/arcep_donnees/mobile/sites/2022_T3/2022_T3_sites_Metropole.csv
NET_WIDTH (km): 17.014
NET_HEIGHT (km): 9.298
SITE_TYPE (type): site_4g
OPERATOR (name): Orange
GEO_AREA (area): Paris
TILE_SIZE (km): 0.025
MU_UE (units): 5
SIGMA_UE (units): 3
POPULATION (people): 11000
NETWORK_DENSITY (people/km^2): 69.5339202013055
OPERATOR_MARKET_SHARE (fraction): 0.385
ACTIVE_UE_FRACTION (fraction): 0.1
UE_MAX_TILE_DISTANCE (tiles): 2
W (MHz): 20
N (dBm/Hz): -100
G0 (linear scale): 3
BS_P_TX (dBm): 25
S (linear scale): 3
ALPHA (linear scale): 2
BS_MAX_RANGE (km): 10


## hints:
https://github.com/nds-group/netmob2023challenge/blob/main/Traffic.ipynb

Technical questions
- how to perfrom handover (https://devopedia.org/5g-handover)
- how to perform paging

To generate UEs: https://colab.research.google.com/github/pymc-devs/pymc-examples/blob/main/examples/gaussian_processes/log-gaussian-cox-process.ipynb

## Simulator

In [ ]:
class UE:
    def __init__(self, ue_id: int, position: Tuple[float, float], tile: int):
        """ 
        Initialize a User Equipment (UE) object.

        Parameters:
        - ue_id (int): The ID of the UE.
        - position (tuple): The position of the UE.
        - tile (int): The tile number of the UE.

        Attributes:
        - ue_id (int): The ID of the UE.
        - position (tuple): The position of the UE.
        - tile (int): The tile number of the UE.
        - active (bool): Indicates if the UE is active.
        - inner_region (bool): Indicates if the UE is in the inner region.
        - bs_id (int): The ID of the base station the UE is connected to.
        - moving (bool): Indicates if the UE is moving.
        - cluster (None or Cluster): The cluster the UE belongs to.
        - generator (None or Generator): The moving generator associated with the UE.
        """
        self.ue_id = ue_id
        self.position = position
        self.tile = tile
        self.active = False 
        self.inner_region = False
        self.bs_id = None
        self.moving = False
        self.cluster = None
        self.generator = None
    
    def get_id(self):
        return self.ue_id

    def set_active(self):
        """ 
        Set the UE as active.
        """
        self.active = True

    def set_inactive(self):
        """ 
        Set the UE as inactive and clear its attributes.
        """
        self.active = False
        # Clear the base station
        # Sanity check
        if self.bs_id is not None: self.bs_id = None
        # Clear variables
        self.inner_region = False
        self.moving = False
        self.generator = None
        self.position = None
        self.tile = None
        self.cluster = None

    def get_info(self):
        """
        Get information about the UE.

        Returns:
        - dict: A dictionary containing the UE's information.
        """
        return {
            'UE ID': self.ue_id,
            'Position': self.position,
            'Tile': self.tile,
            'Active': self.active,
            'Inner Region': self.inner_region, 
            'BS ID': self.bs_id,
            'Moving': self.moving,
            'Generator': self.generator,
            'Cluster': self.cluster  
        }
    
    def ue_trace(self, time):
        """
        Get the trace of the UE.

        Returns:
        - dict: A dictionary containing the UE's location info overtime, in geo-coordinate.
        """
        latitude, longitude = self.get_geo_coordinate()
        x, y = self.position
        return {
            'time': datetime.datetime.now(),
            'id': self.ue_id,
            'latitude': latitude,
            'longitude': longitude,
            'x': x,
            'y': y,
            'cluster': self.cluster,
        }

    def get_geo_coordinate(self):
        """
        Get the geographical coordinates of the UE based on the reference position of origin.

        Returns:
        - tuple: The latitude and longitude of the UE's position.
        """
        # NB
        # source: http://www.edwilliams.org/avform147.htm
        # and https://gis.stackexchange.com/questions/2951/algorithm-for-offsetting-a-latitude-longitude-by-some-amount-of-meters/2980#2980?newreg=0bedc72752ea4e629440a761d6f4a231
        # ORIGIN = (Latitude, Longitude) 
        # self.position = (x, y) in km x=>longitude, y=>latitude

        # Convert latitude and longitude to radians
        lat_rad = math.radians(ORIGIN[0])
        # lon_rad = math.radians(ORIGIN[1])

        # Calculate the change in latitude
        delta_lat = self.position[1] / earth_radius_km

        # Calculate the change in longitude
        delta_lon = self.position[0] / (earth_radius_km * math.cos(lat_rad))

        # Convert the changes to degrees
        delta_lat_deg = math.degrees(delta_lat)
        delta_lon_deg = math.degrees(delta_lon)

        # Calculate the new latitude and longitude
        new_latitude = ORIGIN[0] + delta_lat_deg
        new_longitude = ORIGIN[1] + delta_lon_deg

        return new_latitude, new_longitude
    
    def move(self):
        """
        Move the UE based on the generator associated with the UE's cluster.
        """
        # if not self.generator:
        #     self.generator = gauss_markov_trajectory(position=self.cluster.destination, dimensions=(NET_WIDTH, NET_HEIGHT), group_velocity_mean=self.cluster.group_velocity_mean, 
        #                                      group_theta_mean=self.cluster.group_theta_mean, group_alpha=self.cluster.group_alpha, group_variance=self.cluster.group_variance)
              
        # self.position, _, _ = next(self.generator)
        pass
        #or em_trajectory

In [ ]:
class Cluster():
    def __init__(self, id: int, group_velocity_mean: float =1., group_theta_mean: float =np.pi/2, group_alpha: float =1., group_variance: float =1):
        """
        Initialize a Cluster object.

        Parameters:
        - id (int): The ID of the cluster.
        - group_velocity_mean (float): The mean velocity of the group.
        - group_theta_mean (float): The mean angle of movement of the group.
        - group_alpha (float): The alpha parameter of the group.
        - group_variance (float): The variance parameter of the group.

        Attributes:
        - id (int): The ID of the cluster.
        - group_velocity_mean (float): The mean velocity of the group.
        - group_theta_mean (float): The mean angle of movement of the group.
        - group_alpha (float): The alpha parameter of the group.
        - group_variance (float): The variance parameter of the group.
        - paired_ues (list): The list of paired UEs.
        - destination (tuple): The current destination of the group.
        - generator (None or Generator): The moving generator associated with the group.
        - history (list): The history of destinations visited by the group.
        """
        self.id = id
        self.group_velocity_mean = group_velocity_mean
        self.group_theta_mean = group_theta_mean
        self.group_alpha = group_alpha
        self.group_variance = group_variance
        self.paired_ues = []
        self.destination = (None, None)
        self.generator = None
        self.history = [] #to record trajectory
    
    def get_id(self):
        return self.id

    def resample_params(self):
        """ 
        Resamples the movement parameters of the cluster using uniform distributions. This method adjusts the cluster's velocity, direction, alpha, and variance attributes.
        """
        max_velox = 50 * (1/3600) #km/s. we can set it to 50km/h for urban centers
        min_velox = 5 * (1/3600) #km/s. (assumin 5 km/h which is the average speed of a walk) Going at 5 km/h would take 18 seconds to cover one TILE_SIZE
        self.group_velocity_mean = U(min_velox, max_velox) #defined in km/h
        self.group_theta_mean = U(0, 2*np.pi)
        self.group_alpha = U(0.1, 1)
        self.group_variance = U(0.5, 5)

    def add_ue(self, ue: UE):
        """
        Adds a UE to the cluster's list of paired UEs.
        
        Parameters:
        - ue (UE): The UE object to be added to the cluster.
        """
        self.paired_ues.append(ue)
    
    def remove_ue(self, ue_id: int):
        """
        Removes a UE from the cluster based on its ID.
        
        Parameters:
        - ue_id (int): The ID of the UE to be removed.
        """
        #remove ue with matching UE_id
        for ue in self.paired_ues:
            if ue.ue_id == ue_id:
                self.paired_ues.remove(ue)

    def set_first_destination(self, position: np.ndarray):
        """
        Sets the initial destination for the cluster and initializes the movement generator.
        
        Parameters:
        - destination (tuple): The initial destination coordinates for the cluster.
        """

        self.generator = gauss_markov_trajectory(position = position, dimensions = (NET_WIDTH,NET_HEIGHT), group_velocity_mean=self.group_velocity_mean, 
                                             group_theta_mean=self.group_theta_mean, group_alpha=self.group_alpha, group_variance=self.group_variance)
        #or em_trajectory
        self.move_destination()

    def move_destination(self):
        """
        Updates the cluster's destination using the movement generator and records the new destination.
        """
        self.destination, _ , _ = next(self.generator)
        self.history.append(self.destination)  # Record each new destination

In [ ]:
class BS():
    def __init__(self, controller, bs_id: int, position = Tuple[float, float]):

        """
        Initializes a new instance of the BS (Base Station) class which represents a base station in a cellular network.

        Parameters:
        - bs_id (int): The identifier for the base station.
        - position (tuple): The geographic coordinates of the base station.
        
        Attributes:
        - bw (float): Bandwidth available to the base station.
        - g0 (float): Reference path loss at a distance of 1 meter.
        - p_tx (float): Transmit power of the base station.
        - noise_density (float): Noise density in the network.
        - shadowing (float): Shadowing effect on the signal, specific to each UE.
        - alpha (float): Path loss exponent.
        """

        self.bs_id = bs_id
        self.position = position
        self.cntr = controller
        self.bw= W 
        self.g0 = G0
        self.p_tx = BS_P_TX 
        self.noise_density = N
        self.shadowing = S
        self.alpha = ALPHA
        self.p_tx_in, self.p_tx_out = None, None
        self.served_UEs = [] #memorize the UE object
        self.update_snr_thr = True
        self.snr_thr = None
        self.bs_sector = None
        self.power_adjustment()

    def get_id(self):
        return self.bs_id
    
    def ue_sorting(self):
        """
        Sorts the UEs based on their SNR values, descending order.
        """
        self.served_UEs = sorted(self.served_UEs, key=lambda ue: self.compute_snr(ue), reverse=True)

    def power_adjustment(self, power_levels: Optional[float]= None):
        """
        Adjusts the power output levels for the base station.
        """
        #variables: self.p_tx_in, self.p_tx_out
        #max_cap: BS_P_TX
        #contraints: self.p_tx_out > self.p_tx_in as long as h_out < h_in
        #obj function to max
        if power_levels is None:
            self.p_tx_out = 0.6* BS_P_TX
            self.p_tx_in = BS_P_TX - self.p_tx_out
        else:
            #@TODO be sure to use the right level, you might to apply the median on tuples 
            self.p_tx_in,self.p_tx_out  = median(power_levels, axis = 1) 

    
    #need to implement the alpha-fairness formulation
    def retreive_snr_thr(self, low_complex: bool = False): #as I have not implmented the BO obj function yet, I use a median to compute the snr threshold
        """
        Computes the SNR threshold by taking the median of the SNRs of all served UEs. This is used to determine which UEs are in the inner region of coverage.
        """
        ###
        assert self.served_UEs, 'Empty user set'

        self.ue_sorting() #sort by snr values

        if low_complex:
            snr_lst = [self.compute_snr(ue) for ue in self.served_UEs]
            self.snr_thr = median(snr_lst)
        else:
            #given the ues sorted in discending order, get the partition that yelds the highest value of the objective function. The objective function computes w.r.t the UE's SINR for all the uses served.
            #the partition is the one that maximizes the objective function
            bs = self.cntr.bs_lookup(self.bs_id)
            # prior = float('-inf')
            # ue_trace = []
            # ue_max = None
        
            # for ue in self.served_UEs:
            #     ue_trace.append(ue)
            #     post = self.cntr.objective_function(bss=[bs], ues_test=ue_trace)[0]
            #     if post > prior:
            #         ue_max = ue
            #         prior = post

            ue_trace = []
        
            def objective_for_trace(ue):
                ue_trace.append(ue)
                return self.cntr.objective_function(bss=[bs], ues_test=ue_trace)[0]
        
            ue_max = max(self.served_UEs, key=objective_for_trace)
            self.snr_thr = self.compute_snr(ue_max)
                

    
    def add_ue(self, ue: UE): 
        """
        Adds a UE to the list of UEs served by this base station. Also triggers intra-cell assignment to determine if the UE is within the inner region based on SNR.
        
        Parameters:
        - ue (UE): The user equipment instance to be added.
        """
        self.served_UEs.append(ue)
        self.intra_cell_assignment(ue) 
        self.update_snr_thr = True

    def intra_cell_assignment(self, ue: UE):
        """
        Determines whether the given UE is within the inner region of coverage based on its SNR compared to the SNR threshold.
        
        Parameters:
        - ue (UE): The user equipment instance being evaluated.
        """
        if (self.snr_thr is None) or self.update_snr_thr:
            self.retreive_snr_thr()
            self.update_snr_thr = False
        #agin consider that the real SNR measurement is backpropagated from the UE to the BS, 
        #here we compute at the BS side as it was the DL SNR
        snr_ue = self.compute_snr(ue)
        ue.inner_region = snr_ue > self.snr_thr 
        # if snr_ue > self.snr_thr:
        #     inner_flag = True
        # else:
        #     inner_flag = False
        # ue.inner_region = inner_flag

    def remove_ue(self, ue: UE):
        """
        Removes a UE from the base station's list of served UEs and recalculates the SNR threshold.
        
        Parameters:
        - ue (UE): The user equipment instance to be removed.
        """
        self.served_UEs.remove(ue)
        if self.served_UEs: 
            self.retreive_snr_thr() #update the snr_thr
    
    def get_distance(self, ue:UE):
        """
        Calculates the eucledian distance between the base station and a UE.
        
        Parameters:
        - ue (UE): The user equipment whose distance is to be calculated.
        """
        return math.sqrt(sum((x1 - x2) ** 2 for x1, x2 in zip(self.position, ue.position)))

    def compute_link_gain(self, ue:UE):
        """
        Computes the link gain between the base station and a UE using a standard path loss model.
        
        Parameters:
        - ue (UE): The user equipment for which link gain is computed.
        """
        d = self.get_distance(ue)
        return self.g0 * self.shadowing * (d**(-self.alpha)) #standard path loss model in nominal, LOS conditions

    def compute_snr(self, ue: UE):
        """
        Calculates the Signal-to-Noise Ratio (SNR) for a UE based on its position, either the inner or outer region.
        
        Parameters:
        - ue (UE): The user equipment for which SNR is computed.
        """

        # if ue.inner_region:
        #     pw = self.p_tx_in
        # else:
        #     pw = self.p_tx_out
        # snr = (pw*self.compute_link_gain(ue=ue))/(self.noise_density*self.bw)
        # return snr
        pw = self.p_tx_in if ue.inner_region else self.p_tx_out
        return (pw * self.compute_link_gain(ue)) / (self.noise_density * self.bw)
  
    #eventual functions that allow to modify simulation parameters during the simulation
    def modify_params(self, bw: float = None, g0: float = None,shadowing: float = None,alpha: float =None, p_tx: float = None, noise: float = None):
        """
        Modifies various parameters of the base station dynamically, based on the inputs provided. Each parameter is optional.
        
        Parameters:
        - bw (float): New bandwidth.
        - g0 (float): New reference path loss value.
        - p_tx (float): New transmit power.
        - noise (float): New noise density.
        - shadowing (float): New shadowing effect.
        - alpha (float): New path loss exponent.
        """
        if bw is not None:
            self.bw = bw
        if g0 is not None:
            self.g0 = g0
        if p_tx is not None:
            self.p_tx = p_tx
        if noise is not None:
            self.noise_density = noise
        if shadowing is not None:
            self.shadowing = shadowing
        if alpha is not None:
            self.alpha = alpha

In [ ]:
class Controller:
    def __init__(self):
        """
        Initializes a new instance of the Controller class, which manages user equipment (UEs) and base stations (BSs) in a cellular network system.
        """
        self.UEs = []  # List[UE]
        self.BSs = []  # List[BS]
        self.alpha_fairness = None
        
    def _lookup(self, items: List, identifier: int) -> Optional:
        """
        General-purpose method to retrieve an object from a list using its identifier.
        
        Parameters:
        - items (List): List of items to search through.
        - identifier (int): The identifier of the item to retrieve.
        
        Returns:
        - Optional: The item object if found, None otherwise.
        """
        if items:
            return next((item for item in items if item.get_id() == identifier), None)
        return None

    def bs_lookup(self, bs_id: int = None) -> Optional[BS]:
        """
        Retrieves a base station object from the list of base stations using its identifier.
        
        Parameters:
        - bs_id (int): The identifier of the base station to retrieve.
        
        Returns:
        - Optional[BS]: The base station object if found, None otherwise.
        """
        return self._lookup(self.BSs, bs_id) if bs_id is not None else None
    
    def ue_lookup(self, ue_id: int = None) -> Optional[UE]:
        """
        Retrieves a user equipment object from the list of UEs using its identifier.
        
        Parameters:
        - ue_id (int): The identifier of the user equipment to retrieve.
        
        Returns:
        - Optional[UE]: The user equipment object if found, None otherwise.
        """
        return self._lookup(self.UEs, ue_id) if ue_id is not None else None


    def bs_paging(self):
        """
        Implement paging to locate a UE within the network.
        """
        pass

    # def bs_handover(self, ue: UE):
    #     """
    #     Manages the handover of a UE from one base station to another based on signal-to-noise ratio (SINR) metrics.
    #     - Computes the SINR for all eligible BSs.
    #     - Assigns the UE to the BS with the highest SINR.
    #     - Triggers the intracell assignment process to adjust the UE's service quality or region.

    #     Parameters:
    #     - ue (UE): The user equipment undergoing handover.
    #     """
        
    #     # eligible_bs = copy.deepcopy(self.BSs) #copy of the list of BSs
    #     # current_bs = self.bs_lookup(bs_id = ue.bs_id)
    #     # # current_bs = None

    #     # # if ue.bs_id is not None:
    #     # #     current_bs = self.bs_lookup(bs_id = ue.bs_id)
    #     # #     eligible_bs.remove(current_bs)

    #     # if (current_bs is None) or (current_bs.get_distance(ue) >= BS_MAX_RANGE): #change BS
            
    #     #     if current_bs is not None:
    #     #         eligible_bs.remove(current_bs)
    #     #         current_bs.remove_ue(ue)
            
    #     #     max_sinr_ix = argmax(np.array([self.compute_sinr(bs, ue) for bs in eligible_bs])) #that would be the UL SINR, for pairing

    #     #     selected_bs = eligible_bs[max_sinr_ix]

    #     #     selected_bs.add_ue(ue) #==> trigger intracell assignment
    #     #     ue.bs_id = selected_bs.bs_id

    #     # if ue.moving:
    #     #     ue.moving = False
    #     ue_in_range = True

    #     current_bs = self.bs_lookup(ue.bs_id)

    #     eligible_bs = [bs for bs in self.BSs if bs.get_distance(ue) < BS_MAX_RANGE] 

    #     if current_bs is None or current_bs.get_distance(ue) >= BS_MAX_RANGE:

    #         if current_bs is not None:
    #             current_bs.remove_ue(ue)
            
    #         if not eligible_bs:
    #             print("User not reachable from BSs")
    #             print(f'Kill user {ue.ue_id}')
    #             ue_in_range = False# Early exit if no BS is eligible

    #         current_bs = max(eligible_bs, key=lambda bs: self.compute_sinr(bs, ue, eligible_bs))

    #         current_bs.add_ue(ue) #here will trigger the intra-cell region assignment
            
    #         ue.bs_id = current_bs.bs_id

    #     ue.moving = False

    #     self.compute_sinr(bs = current_bs, ue=ue, eligible_bs = eligible_bs) #update the sinr metrics

    #     return ue_in_range

    # def compute_sinr(self, bs: BS, ue: UE, eligible_bs: List[BS])->float:
    #     """
    #     Computes the downlink SINR for a given UE at a specified base station.
        
    #     Parameters:
    #     - bs (BS): The base station.
    #     - ue (UE): The user equipment.
        
    #     Returns:
    #     - float: The computed SINR.
    #     """

    #     dl_sinr = None 

    #     ue_just_spawned = ue.dl_sinr is None

    #     pairing_flag = ue.moving or ue_just_spawned

    #     bs_interference_set = [b for b in eligible_bs if b != bs]

    #     noise = bs.bw * bs.noise_density

    #     inter_cell_interf = sum([b.compute_link_gain(ue)*(b.p_tx_in + b.p_tx_out) for b in bs_interference_set]) #in the case some BS do not have the same power or gain

    #     interference = noise + inter_cell_interf

    #     link_gain = bs.compute_link_gain(ue)

    #     if pairing_flag or not ue.inner_region:

    #         intra_cell_interf = link_gain*bs.p_tx_in

    #         interference += intra_cell_interf 

    #         dl_sinr = (link_gain*bs.p_tx_out)/interference
    #     else:
    #         dl_sinr = (link_gain*bs.p_tx_in)/interference
        
    #     if not pairing_flag:
    #         ue.dl_sinr = dl_sinr
    #         print('UE_ID: ', ue.ue_id, ' SINR: ', ue.dl_sinr)
    #     # print(link_gain*bs.p_tx_in)
    #     # print(interference)

    #     return dl_sinr
    def bs_handover(self, ue: UE):
        """
        Manages the handover of a UE from one base station to another based on signal-to-noise ratio (SINR) metrics.
        - Computes the SINR for all eligible BSs.
        - Assigns the UE to the BS with the highest SINR.
        - Triggers the intracell assignment process to adjust the UE's service quality or region.

        Parameters:
        - ue (UE): The user equipment undergoing handover.
        """
        
        # eligible_bs = copy.deepcopy(self.BSs) #copy of the list of BSs
        # current_bs = self.bs_lookup(bs_id = ue.bs_id)
        # # current_bs = None

        # # if ue.bs_id is not None:
        # #     current_bs = self.bs_lookup(bs_id = ue.bs_id)
        # #     eligible_bs.remove(current_bs)

        # if (current_bs is None) or (current_bs.get_distance(ue) >= BS_MAX_RANGE): #change BS
            
        #     if current_bs is not None:
        #         eligible_bs.remove(current_bs)
        #         current_bs.remove_ue(ue)
            
        #     max_sinr_ix = argmax(np.array([self.compute_sinr(bs, ue) for bs in eligible_bs])) #that would be the UL SINR, for pairing

        #     selected_bs = eligible_bs[max_sinr_ix]

        #     selected_bs.add_ue(ue) #==> trigger intracell assignment
        #     ue.bs_id = selected_bs.bs_id

        # if ue.moving:
        #     ue.moving = False
        def subroutine(ue:UE, eligible_bs: List[BS]):
            bs = max(eligible_bs, key=lambda bs: self.compute_sinr(bs = bs, ue = ue, interference_set=eligible_bs, pairing= True))
            bs.add_ue(ue) #here will trigger the intra-cell region assignment
            ue.bs_id = bs.get_id()
            #ue.dl_sinr = self.compute_sinr(bs, ue, interference_set)

        ue_in_range = True

        current_bs = self.bs_lookup(ue.bs_id)

        ue_just_spawned = current_bs is None

        eligible_bs = [b for b in self.BSs if b.get_distance(ue) < BS_MAX_RANGE]

        if not eligible_bs:
                print("User not reachable from BSs")
                print(f'Kill user {ue.ue_id}')
                ue_in_range = False# Early exit if no BS is eligible

        elif ue_just_spawned:
            subroutine(ue,eligible_bs)
        
        elif ue.moving and current_bs.get_distance(ue) >= BS_MAX_RANGE: 
                
                current_bs.remove_ue(ue)
                ue.moving = False
                subroutine(ue, eligible_bs)
        #else:
            #subroutine(current_bs, ue, eligible_bs)
        #print('UE_ID: ', ue.ue_id, ' SINR: ', ue.dl_sinr)

        return ue_in_range
        

    def compute_sinr(self, bs: BS, ue: UE, pairing:bool, interference_set: List[BS] = None, inner_test: bool = False)->float:
        """
        Computes the downlink SINR for a given UE at a specified base station.
        
        Parameters:
        - bs (BS): The base station.
        - ue (UE): The user equipment.
        
        Returns:
        - float: The computed SINR.
        """

        dl_sinr = .0 

        if interference_set is None:
            bs_interference_set = [b for b in self.BSs if b.get_distance(ue) < BS_MAX_RANGE]
        else:
            bs_interference_set = [b for b in interference_set if b!=bs]

        noise = bs.bw * bs.noise_density

        inter_cell_interf = sum([b.compute_link_gain(ue)*(b.p_tx_in + b.p_tx_out) for b in bs_interference_set]) #in the case some BS do not have the same power or gain

        interference = noise + inter_cell_interf

        link_gain = bs.compute_link_gain(ue)

        if pairing:

            intra_cell_interf = link_gain*bs.p_tx_in

            interference += intra_cell_interf 

            dl_sinr = (link_gain*bs.p_tx_out)/interference

        elif ue.inner_region or inner_test:
              
              dl_sinr = (link_gain*bs.p_tx_in)/interference

        else:
            intra_cell_interf = link_gain*bs.p_tx_in

            interference += intra_cell_interf 

            dl_sinr = (link_gain*bs.p_tx_out)/interference

        return dl_sinr
    
    def kill_ue(self, ue: UE):
        """
        Removes a UE from the system entirely, including dissociation from the BS and marking the UE as inactive.
        
        Parameters:
        - ue (UE): The user equipment to be removed.
        """
        current_bs = self.bs_lookup(bs_id = ue.bs_id)
        current_bs.remove_ue(ue)
        ue.set_inactive() #==> here it will dissociate from the bs
        self.UEs.remove(ue)

    def add_ue(self, ue:UE):
        """
        Adds a new UE to the system, sets it as active, and performs an initial base station assignment.
        
        Parameters:
        - ue (UE): The user equipment to be added.
        """
        self.UEs.append(ue)
        ue.set_active()
        return self.bs_handover(ue)

    def gather_metrics(self):
        """
        Collects and returns the SINR metrics for each UE served by its BS in the network.
        
        Returns:
        - dict: {'median_sinr': float, 'average_sinr': float} the median and average SINR across the network.
        """
        metrics_ = [self.compute_sinr(bs=self.bs_lookup(ue.bs_id), ue=ue, pairing=False) for ue in self.UEs]
        # for ue in self.UEs:
        #     bs_metrics = [ue.dl_sinr for ue in bs.served_UEs]
        #     #print(bs)
        #     #print(bs.served_UEs)
        #     #print(bs_metrics)
        #     metrics_.extend(bs_metrics)
        rounder = lambda x: round(x,5)
        return {'median_sinr': rounder(median(metrics_)), 'average_sinr': rounder(mean(metrics_))}


    def objective_function(self, bss: List[BS], ues_test: List[UE] = [], alpha_fairness: float = 1.0):
        """
        Defines the objective function for the base station.
        """
        alpha_fairness = self.alpha_fairness

        normalizer = lambda bs, lst: np.array(lst)/np.sum(lst) if alpha_fairness != 0 else [1 if ue == bs.served_UEs[argmax(np.array(lst))] else 0 for ue in bs.served_UEs]

        objective_func = lambda lst: np.sum(np.log(np.array(lst))) if alpha_fairness == 1 else np.sum((np.array(lst)**(1-alpha_fairness))/(1-alpha_fairness))

        shannon_capacity = lambda bs, ue, ue_inner_test : bs.bw * math.log(1 + self.compute_sinr(bs=bs, ue=ue, pairing=False, inner_test=ue_inner_test),2)

        scheduler = lambda bs, ue, ue_inner_test:  (shannon_capacity(bs, ue, ue_inner_test)**(1-alpha_fairness))/ alpha_fairness if alpha_fairness != 0 else shannon_capacity(bs, ue, ue_inner_test)
        
        lister = lambda bs: [scheduler(bs, ue, ue in ues_test) for ue in bs.served_UEs]

        return [objective_func(normalizer(bs, lister(bs))) for bs in bss]

    
        # return objective_func(scheduling, alpha_fairness)
    def set_sectors(self):
         #supposing all the BSs have the same MAX range
        for bs in self.BSs:
            #get the interference set of the base station
            bs.sector = [b for b in self.BSs if b != bs and bs.get_distance(b) < 2*BS_MAX_RANGE]
    
    def optimize_power(self):
        """
        Implements a consensus algorithm to adjust the power levels of the base stations withing a common sector.
        """
        for bs in self.BSs:
            #get the recoomendation from the sector
            #trigger optimization
            bs.power_adjstument(self.objective_function(bss = bs.sector))

    def build_ues_db(self, current_dataset: list, time = None, filename: Optional[str] = None):
        """
        Builds and exports a database of UEs with their current information to a CSV file.
        
        Parameters:
        - filename (str): The path to the CSV file where data will be stored.
        """ 
        if filename is not None:
            ues_db = pd.DataFrame(current_dataset)
            ues_db.to_csv(filename, index=False)
        else:
            assert time is not None, 'Time must be provided'
            data = [ue.ue_trace(time=time) for ue in self.UEs]
            current_dataset.extend(data)
                #each ue will have the list of coordinates it visited

In [ ]:
class MemoryHandler(logging.Handler):
    def __init__(self):
        super().__init__()
        self.records = []

    def emit(self, record):
        self.records.append(self.format(record))

class JsonFormatter(logging.Formatter):
    def format(self, record):
        message = record.msg
        if isinstance(message, dict):
            return json.dumps(message)
        return json.dumps({"message": message})

In [ ]:
#following the process used in "Assessing the Performance of NOMA in a Multi-Cell Context: A General Evaluation Framework"
class SIMULATION():
    """
    A simulation framework for evaluating the performance of NOMA in a cell-free context, 
    based on dynamic user equipment (UE) behavior and base station (BS) interaction.
    """ 

    def __init__(self, controller: Controller, id:int=1, prob_map: List = None,tau: float = 1, lambda_: float = 2, mu: float = 4, epsilon: float = 0.4, move_interval: int = 3): #rho = lambda/mu < 1
        """
        Initializes the simulation environment, sets parameters for the simulation, and optionally, an initial probability map for UE distribution across tiles.

        Parameters:
        - controller (Controller): The control unit handling all UEs and BSs in the simulation.
        - prob_map (list, optional): A list indicating the initial probability of UE distribution across tiles.
        - tau (float): Interval for collecting SINR metrics.
        - lambda_ (float): Rate of UE arrivals.
        - mu (float): Rate of UE departures.
        - epsilon (float): Probability of exploration in UE movement.
        - move_interval (int): Interval in seconds between each movement update for UEs.

        Attributes:
        - n_tiles (int): Number of tiles across the simulated area.
        - num_tiles_x (int): Number of tiles along the width of the simulated area.
        - num_tiles_y (int): Number of tiles along the height of the simulated area.
        - env (simpy.Environment): The simulation environment from SimPy.
        - history_logger (logging.Logger): Logger for recording simulation history and metrics.
        """
        self.id = id
        self.controller = controller
        self.ues_traces = []
        #self.n_tiles = int((NET_WIDTH*NET_HEIGHT)// (TILE_SIZE**2))
        self.num_tiles_x = int(NET_WIDTH // TILE_SIZE)
        self.num_tiles_y = int(NET_HEIGHT // TILE_SIZE)
        self.n_tiles = self.num_tiles_x * self.num_tiles_y
        self.env = simpy.Environment()
        self.tau = tau  # Interval for SINR metrics collection in seconds
        self.lambda_ = lambda_  # User birth rate
        self.mu = mu  # User lifetime rate parameter
        self.move_interval = move_interval  # Time interval for user movement
        self.epsilon = epsilon  # Probability of exploration in movement
        if prob_map is not None:
            self.prob_map = prob_map
        else:
            self.prob_map_random_initialization()
        self.clusters = None

        #run the simulation
        # Set up a logging system to collect time and SINR metrics
        #logging.basicConfig(level=logging.INFO)
        
        #logging.basicConfig(format="%(levelname)s | %(asctime)s | %(message)s")
        
        self.history_logger = logging.getLogger('history')
        # stdout = logging.StreamHandler(stream=sys.stdout)
        # stdout.setLevel(logging.INFO)
        # self.history_logger.addHandler(stdout)
        # self.history_logger.setLevel(logging.INFO)

        memory_handler = MemoryHandler()
        memory_handler.setLevel(logging.INFO)
        formatter = JsonFormatter()
        memory_handler.setFormatter(formatter)
        self.history_logger.addHandler(memory_handler)
        self.history_logger.setLevel(logging.INFO)
        self.history_logger.propagate = False

    def prob_map_random_initialization(self):
        """
        Initializes the probability map randomly if no predefined map is provided.
        """
        raw_probabilities = [random.random() for _ in range(self.n_tiles)]
        total = sum(raw_probabilities)
        self.prob_map = [prob / total for prob in raw_probabilities]


    def sample_position(self, tile_loc: Optional[int] = None, old_position: Optional[Tuple[float, float]] = None) -> Tuple[float, float]:
        """
        Samples a uniform random position within a given tile for placing a UE.
        """
        def random_offset() -> float:
            """Generates a random offset within the tile size."""
            return U(0, TILE_SIZE)

        if tile_loc is not None:
            row_index = tile_loc // self.num_tiles_x
            col_index = tile_loc % self.num_tiles_x

            tile_base_x = col_index * TILE_SIZE
            tile_base_y = row_index * TILE_SIZE

            new_position = (tile_base_x + random_offset(), tile_base_y + random_offset())
        elif old_position is not None:
            new_position = (old_position[0] + random_offset(), old_position[1] + random_offset())
        else:
            raise ValueError("Either tile_loc or old_position must be specified.")

        return new_position

    def sample_ue(self):
        """
        Randomly selects a UE from the controller's list of UEs.
        """
        return np.random.choice(self.controller.UEs)
        
    def sample_tile(self, tile_loc: int= None):
        """
        Samples a tile based on the probability map for new UE birth or based on a specific range for movement.
        """
        if tile_loc is None:
            ret = np.random.choice(range(self.n_tiles), p=self.prob_map)
        else: #given a tile location, sample a new tile within the reachable tiles
            low_bnd = max(0, tile_loc - UE_MAX_TILE_DISTANCE)
            up_bnd = min(tile_loc + UE_MAX_TILE_DISTANCE, self.n_tiles - 1)
            ret = np.random.randint(low_bnd,up_bnd)

        return ret

    def generate_bs(self, filepath: str):
        """
        Loads BS data from a CSV file and initializes BS objects within the controller.
        """
        bs_df = pd.read_csv(filepath)
        #self.controller.BSs = [BS(bs_id = bs_df.loc[i].id_station_anfr, position = (bs_df.loc[i].x,bs_df.loc[i].y))for i in bs_df.index] #position is in km from origin
        self.controller.BSs = [BS(bs_id=row.id, position=(row.x, row.y), controller = self.controller) for _, row in bs_df.iterrows()]
        self.controller.set_sectors()
        #logging.info(f'Number of Base Stations: {len(self.controller.BSs)}')
        #self.controller.BSs = self.controller.BSs[:300]
        print('num_BSs: ', len(self.controller.BSs))
    
    #the first pool of user can be sampled using a Log Gaussian Cox process
    def generate_UEs(self, max_cap_: int = None, verbose: bool = False):
        """
        Generates an initial population of UEs based on a log-Gaussian Cox process.
        """
        # Sample a Gaussian random variable for each tile
        gaussian_samples = np.random.normal(MU_UE, SIGMA_UE, self.n_tiles)

        # Calculate intensity and sample the number of UEs per tile
        intensity = np.exp(gaussian_samples)
        ue_counts = np.random.poisson(intensity)

        # Generate UEs uniformly within each tile
        users = []
        ue_id = 1
        ###
        if max_cap_ is not None and max_cap_ <= self.n_tiles:
            sampled_tiles = np.random.choice(self.n_tiles, max_cap_, replace=False)
        else:
            sampled_tiles = range(self.n_tiles)  # Use all tiles if max_cap_ is None or too large

        if verbose: print("Sampled Tiles: ",sampled_tiles)

        for tile_loc in sampled_tiles:
            if verbose: 
                print("Tile: ",tile_loc)
                print("Number of users to be generated: ", range(ue_counts[tile_loc]))
            for _ in range(ue_counts[tile_loc]):
                new_ue = UE(ue_id = ue_id, position = self.sample_position(tile_loc = tile_loc), tile = tile_loc)
                users.append(new_ue)
                ue_id += 1

        if verbose: print("Generated UEs: ", len(users))     
        # Select UEs based on Operator market share
        selected_active_users = np.random.choice(users, int(len(users) * OPERATOR_MARKET_SHARE * ACTIVE_UE_FRACTION), replace=False)
        # Set "ACTIVE_UE_FRACTION" of the selected UEs as active
        for ue in selected_active_users: 
            ue.set_active()
        self.controller.UEs =  selected_active_users
        print('num_active_users: ', len(self.controller.UEs))
    
    def generate_random_ues(self, total_population: int, verbose: bool = False):
        """
        Generates a random population of UEs distributed across tiles based on the self.prob_map.
        Args:
            total_population (int): The total number of UEs to generate.

        This function will generate `total_population` UEs, distribute them across tiles based
        on the probability map, and then place each UE at a random position within its assigned tile.
        """
        # Sample the tiles for each UE based on the probability map
        tiles = np.random.choice(range(self.n_tiles), size=total_population, p=self.prob_map)

        # Generate UEs within their respective tiles
        for tile in tiles:
            # Sample a position within the tile
            position = self.sample_position(tile_loc = tile)

            # Create a new UE with a unique ID
            ue_id = len(self.controller.UEs) + 1  # Unique ID for each UE
            new_ue = UE(ue_id=ue_id, position=position, tile=tile)
            if not self.controller.add_ue(ue = new_ue):
                self.ue_death(ue= new_ue)

        #if verbose:  # Assuming there is a verbose attribute for debug information
        print('num_active_users: ', len(self.controller.UEs))

    
    def ue_arrival(self, time: float = None): #randomly placing inside the tile (tile_loc is the number of which tile inside the grid)
        """
        Simulates the arrival of new UEs over time, integrating them into existing clusters.
        """
        if time is not None:
            yield self.env.timeout(time)

        ue_id = self.controller.UEs[-1].ue_id + 1 if self.controller.UEs else 1
        tile_loc = self.sample_tile()
        new_ue = UE(ue_id = ue_id, position = self.sample_position(tile_loc = tile_loc), tile = tile_loc)
        #new_ue.set_active()
        if not self.controller.add_ue(ue = new_ue):  
            yield self.env.process(self.ue_death(new_ue, 0))

        c_ix = np.argmin(np.array([l2_norm(np.array(new_ue.position) - np.array(c.destination)) for c in self.clusters]))
        self.clusters[c_ix].add_ue(ue = new_ue)
        new_ue.cluster = self.clusters[c_ix].get_id()
        # Add the user to the controller
        #self.controller.add_ue(ue = new_ue) ##chose bs to pair based on snr level

        # Schedule user death after a lifetime
        lifetime = random.expovariate(self.mu)
        self.env.process(self.ue_death(new_ue, lifetime))

        # Schedule the next user arrival
        inter_arrival_time = random.expovariate(self.lambda_)
        self.env.process(self.ue_arrival(time = inter_arrival_time))

    def ue_translation(self):
        """
        The clusters movement happens accordint to an epsilon greedy strategy, then UEs moves following the cluster destination. 
        The BS handover function checks whether the displacement has exceeded the maximum coverage radius of the current base station, before triggering a new BS pairing.
        """
        while True:
            for c in self.clusters:
                if random.random() < (1 - self.epsilon):
                    c.move_destination()
                    # Trigger user movement within the cluster
                    for ue in c.paired_ues:
                        ue.moving = True
                        ##depending on the mobility_model used
                        ue.position = self.sample_position(old_position= c.destination)
                        #ue.move() #if the displacement is above a given threshold, the handover will be triggered
                        if not self.controller.bs_handover(ue = ue):
                            yield self.env.process(self.ue_death(ue, 0))

                    self.controller.build_ues_db(current_dataset=self.ues_traces, time = self.env.now) #problema se il displacement di uno ue va fuori dai bordi, viene killato subito

            yield self.env.timeout(self.move_interval)


    def metrics_collection(self):
        """
        Periodically collects SINR metrics across all BS-UE pairs in the simulation.
        """
        while True:
            #self.controller.optimize_power()
            sinr_metrics = self.controller.gather_metrics()
            self.history_logger.info({
                'time': self.env.now,
                **sinr_metrics  # add also hyper-params
            })
            yield self.env.timeout(self.tau)

    def ue_death(self, ue: UE, lifetime: float = None):
        """
        Simulates the departure of a UE after its lifetime expires.
        """
        if lifetime is not None:
            yield self.env.timeout(lifetime)
        if ue.active:
            cluster = self.controller._lookup(self.clusters, ue.cluster)
            cluster.remove_ue(ue_id=ue.ue_id)
            self.controller.kill_ue(ue)
        
    # def ue_translation(self,ue: UE):
    #     #change coordinates of UE
    #     #trigger controller handover
    #     #ue can move up to MaxDistance tiles from the current tile
    #     ue.moving = True
    #     low_bnd = ue.tile-UE_MAX_TILE_DISTANCE if ue.tile>UE_MAX_TILE_DISTANCE else ue.tile-ue.tile 
    #     up_bnd = ue.tile+UE_MAX_TILE_DISTANCE
    #     new_tile_loc = np.random.randint(low_bnd,up_bnd)
    #     ue.position = self.sample_position(tile_loc = new_tile_loc)
    #     self.controller.bs_handover(ue = ue) #pair with the new bs in the new location

    def cluster_UEs(self, n_clusters):
        """
        Cluster UEs based on their spatial positions using K-means clustering.
        
        Parameters:
            users (list of UE): List of user equipment (UE) objects.
            n_clusters (int): Number of clusters to create.
        
        Returns:
            list of Cluster: List of Cluster objects with UEs grouped.
        """
        # Extract user positions from UE objects
        users = self.controller.UEs
        user_positions = np.array([ue.position for ue in users])

        # Apply K-means clustering
        kmeans = KMeans(n_clusters=n_clusters, n_init='auto').fit(user_positions)

        # Create Cluster objects based on the number of clusters
        self.clusters = [Cluster(id = i) for i in range(n_clusters)]
        #clusters = [Cluster(id = i) for i in range(len(np.unique(kmeans.labels_)))]
        
        for i,cluster in enumerate(self.clusters):
            cluster.resample_params()
            cluster.set_first_destination(position= kmeans.cluster_centers_[i])

        # Assign cluster labels to UEs and add them to their corresponding Cluster
        for ue, label in zip(users, kmeans.labels_):
            ue.cluster = label
            self.clusters[label].add_ue(ue)
    
    def write_log(self):
        # Ensure output directory exists
        os.makedirs('output', exist_ok=True)
        with open(f'output/simulation_logs_sim{self.id}.json', 'w') as log_file:
            for record in self.history_logger.handlers[0].records:
                log_file.write(record + '\n')

    def initialize(self):
        pass

    def run(self, filepath: str, max_cap_: int = None, verbose: bool = False, n_clusters: int = 5, separate_plots: bool = True , gif_toggle: bool = False, alpha_fairness: float = 1.0): #update description
        """
        Runs the entire simulation setup, managing UE births, movements, and SINR collection.
        """

        print('network_area: ', round(NET_WIDTH*NET_HEIGHT,3), f'{DISTANCE_METRIC}^2')
        print('n_tiles: ', self.n_tiles, f'({TILE_SIZE}', f'{DISTANCE_METRIC})')

        self.controller.alpha_fairness = alpha_fairness

        #Populate the network with base stations
        self.generate_bs(filepath)

        # Intial static user population of the network
        #self.generate_UEs(max_cap_=max_cap_, verbose=verbose)
        self.generate_random_ues(total_population=max_cap_)

        # #plot the network
        # self.plot_net()

        # # Build the UEs database
        # self.controller.build_ues_db(filename= "ues_set.csv")

        #cluster the UES
        self.cluster_UEs(n_clusters)
        #self.cluster_UEs(k_means_cluster) #hparam
        # Start the birt-death process
        self.env.process(self.ue_arrival())

        # Start the cluster movement process
        self.env.process(self.ue_translation())

        # Start the SINR collection process
        self.env.process(self.metrics_collection())

        for t in tqdm(range(SIM_TIME)):
            self.env.step()

        self.controller.build_ues_db(filename=f'output/ues_traces_sim{self.id}.csv', current_dataset=self.ues_traces)
        self.plot_net2D()
        self.plot_metrics_history(separate_plots=separate_plots)
        self.write_log()
            # if gif_toggle:
            #     map_update_ue_positions(map_obj, self.controller.UEs)  # Update UEs on the map
            #     filename = f"snapshot_{t}.html"
            #     save_map_state(map_obj, filename)

        # if gif_toggle:
        #     draw_cluster_trajectories(map_obj, clusters)  # Update trajectories on the map
        #     create_gif('path_to_snapshots_folder', 'simulation.gif', duration=1)

        # Run the SimPy event loop
        

        #to model the time 
        
        #poisson process to spwan and to kill users
        
        # Start the first user arrival
        


        #simulate timeline
        # T = 100 #s
        # ues_pool = generate_user(n_users, prob_map, grid) #==>return of class UEs 
        # generate_ues_clusters(n_clusters) #==> pair each user with a label indicating the cluster,
        # generate_cluster_params(n_clusters) #==> return a dict: key cluster_id, parameters for each cluster
        # for t in range(T):
        #     for c in clusters:
        #         for ue in c.paired_ues:
        #             ue.move()
        #         c.move_destination()

        #     if t%T_SAMPLE == 0:
        #         compute_analytics()
        #         update_log()

        # print_log()
        # plot_simulation2D()

        # if graphics:
        #     plot_simulation3D()


    
    def plot_net2D(self):
        """
        Visualizes the 2D layout of the network, including UEs, BSs, and clusters.
        """

        bss_net = self.controller.BSs
        ues_net = self.controller.UEs
        km_to_m = 1000

        fig, ax = plt.subplots()

        # Plot end-users and base stations
        ue_x_positions = [ue.position[0]*km_to_m for ue in ues_net]
        ue_y_positions = [ue.position[1]*km_to_m for ue in ues_net]
        
        ax.scatter(ue_x_positions, ue_y_positions, marker='.', c='gray', alpha=0.75, s=1, label='End-user')

        bs_x_positions = [bs.position[0]*km_to_m for bs in bss_net]
        bs_y_positions = [bs.position[1]*km_to_m for bs in bss_net]

        ax.scatter(bs_x_positions, bs_y_positions, marker='^', c='orange',s=10, label='Base Station')

        # Draw a square region of interest
        #ax.add_patch(plt.Rectangle((1500, 1500), 2000, 2000, fill=False, edgecolor='blue', linewidth=2))

        # Draw the boundary of the whole area
        #ax.add_patch(plt.Rectangle((0, 0), 5000, 5000, fill=False, edgecolor='red', linewidth=2))

        # Plot each cluster's trajectory
        for cluster in self.clusters:
            # Ensure that the cluster has a non-empty history to plot
            if hasattr(cluster, 'history') and cluster.history:
                x_positions = [pos[0] * km_to_m for pos in cluster.history]
                y_positions = [pos[1] * km_to_m for pos in cluster.history]
                ax.plot(x_positions, y_positions, marker='', linestyle='-', alpha=0.5, label=f'Cluster {cluster.id}')

        # Labeling the axes
        ax.set_xlabel('Distance (m)')
        ax.set_ylabel('Distance (m)')

        # Set the plot limits
        ax.set_xlim(0, NET_WIDTH*km_to_m)
        ax.set_ylim(0, NET_HEIGHT*km_to_m)

        # Add a legend
        if len(self.clusters) < 5:
            ax.legend(loc = 'upper right')

        #save the plot as png image in the folder output
        plt.savefig(f'output/2D_network_sim{self.id}.png')

        # Show the plot
        plt.show()


    # def plot_metrics_history(self):
    #     """
    #     Plot the history of collected metrics over time.
    #     This function visualizes metrics such as SINR over the course of the simulation.
    #     """
    #     times, metrics = zip(*[(log['time'], log['sinr']) for log in self.history_logger.handlers[0].records])
    #     plt.figure(figsize=(10, 5))
    #     for idx, metric_snapshot in enumerate(metrics):
    #         plt.plot(times, metric_snapshot, label=f'Metric at {idx}')
    #     plt.xlabel('Simulation Time')
    #     plt.ylabel('Metrics Value')
    #     plt.title('Metrics Over Time')
    #     plt.legend()
    #     plt.show()

    # def plot_metrics_history(self, separate_plots=True):
    #     """
    #     Plot the history of collected metrics over time.
    #     This function visualizes various metrics collected during the simulation. It can generate either a single plot with all metrics or separate plots for each metric.

    #     Args:
    #     separate_plots (bool): If True, generate separate plots for each metric.
    #     """
    #     # Check if there is data to plot
    #     if not self.history_logger.handlers[0].records:
    #         print("No data to plot.")
    #         return

    #     # Gather times and all metrics from the records
    #     times = [log['time'] for log in self.history_logger.handlers[0].records]
    #     metric_keys = set(key for log in self.history_logger.handlers[0].records for key in log.keys() if key != 'time')
    #     hyper_params = [key for key in metric_keys if '_hyper_param' in key]
    #     metrics_to_plot = [key for key in metric_keys if '_hyper_param' not in key]

    #     if separate_plots:
    #         # Plot each metric on a separate figure
    #         for metric in metrics_to_plot:
    #             plt.figure(figsize=(10, 5))
    #             metric_values = [log[metric] for log in self.history_logger.handlers[0].records]
    #             plt.plot(times, metric_values, label=f'{metric.capitalize()}')
    #             hyper_param_values = ', '.join([key.split('_hyper_param')[0].capitalize() for key in hyper_params])
    #             plt.xlabel('Simulation Time')
    #             plt.ylabel('Metric Values')
    #             plt.title(f'{metric.capitalize()} Over Time')
    #             if hyper_param_values:
    #                 plt.legend([f'{metric.capitalize()} ({hyper_param_values})'])
    #             else:
    #                 plt.legend()
    #             plt.show()
    #     else:
    #         # Plot all metrics on one figure
    #         plt.figure(figsize=(10, 5))
    #         for metric in metrics_to_plot:
    #             metric_values = [log[metric] for log in self.history_logger.handlers[0].records]
    #             plt.plot(times, metric_values, label=f'{metric.capitalize()}')
    #         hyper_param_values = ', '.join([key.split('_hyper_param')[0].capitalize() for key in hyper_params])
    #         plt.xlabel('Simulation Time')
    #         plt.ylabel('Metric Values')
    #         plt.title('Metrics Over Time')
    #         if hyper_param_values:
    #             plt.legend([f'{metric.capitalize()} ({hyper_param_values})' for metric in metrics_to_plot])
    #         else:
    #             plt.legend()
    #         plt.show()

    def plot_metrics_history(self, separate_plots=True):
        handler = self.history_logger.handlers[0]
        if not hasattr(handler, 'records') or not handler.records:
            print("No data to plot.")
            return

        # Parse records assuming they are in JSON format
        records = [json.loads(record) for record in handler.records]  # Adjusting how records are parsed
        times = [log['time'] for log in records]
        metric_keys = {key for log in records for key in log if key != 'time'}

        if separate_plots:
            for metric in metric_keys:
                plt.figure(figsize=(10, 5))
                metric_values = [log[metric] for log in records if metric in log]  # Safety check for key existence
                plt.plot(times, metric_values, label=f'{metric.capitalize()}')
                plt.xlabel('Time (s)')
                plt.ylabel('Metric Values')
                plt.title(f'{metric.capitalize()} Over Time')
                plt.legend()
                plt.savefig(f'output/{metric}_sim{self.id}.png')
                plt.show()
        else:
            plt.figure(figsize=(10, 5))
            for metric in metric_keys:
                metric_values = [log[metric] for log in records if metric in log]  # Safety check for key existence
                plt.plot(times, metric_values, label=f'{metric.capitalize()}')
            plt.xlabel('Time (s)')
            plt.ylabel('Metric Values')
            plt.title('Metrics Over Time')
            plt.legend()
            plt.savefig(f'output/metrics_sim{self.id}.png')
            plt.show()


SIM_TIME = 500
n_clusters = 5
cntr = Controller()
sim = SIMULATION(cntr)
sim.run(id= 1, filepath = "simulation_datasets/Filtered_4G_Sites_Orange_Paris.csv",n_clusters=n_clusters, max_cap_= 600, verbose = False)